In [ ]:
%run react_functions.ipynb

import shutil
import os

if shutil.which("g16"):
    print("✅ 环境检查通过：系统已找到 Gaussian (g16)。")
else:
    print("❌ 警告：未找到 g16，请检查环境变量配置。")

import py3Dmol
from rdkit import Chem

def view_with_labels(mol, name):
    """专门用于显示原子序号的可视化函数"""
    mb = Chem.MolToMolBlock(mol)
    view = py3Dmol.view(width=600, height=600)
    view.addModel(mb, 'mol')
    view.setStyle({'stick': {'radius': 0.15}, 'sphere': {'scale': 0.25}})
    view.setBackgroundColor('0xeeeeee')
    
    # 添加原子序号标签
    for i, atom in enumerate(mol.GetAtoms()):
        pos = mol.GetConformer().GetAtomPosition(i)
        view.addLabel(str(i+1), {
            'position': {'x': pos.x, 'y': pos.y, 'z': pos.z},
            'backgroundColor': 'white',
            'fontColor': 'black',
            'fontSize': 14,
            'backgroundOpacity': 0.8
        })
    
    view.zoomTo()
    view.show()
    print(f"=== {name} (带原子序号) ===")

In [ ]:
print(">>> 1. 构建 NH3 + MeI 体系...")

# --- 1. 手动构建分子 (不依赖随机生成) ---
mol_nh3 = Chem.MolFromSmiles("N") 
mol_nh3 = Chem.AddHs(mol_nh3)
# NH3 也是 4 个原子，先 Embed
AllChem.EmbedMolecule(mol_nh3, randomSeed=42)

mol_mei = Chem.MolFromSmiles("CI") 
mol_mei = Chem.AddHs(mol_mei)
AllChem.EmbedMolecule(mol_mei, randomSeed=42)

# --- 2. 几何对齐 (让孤对电子对准 C) ---
conf_nh3 = mol_nh3.GetConformer()
conf_mei = mol_mei.GetConformer()

# A. 摆放 NH3 (N 在原点，H 在 Z 轴负方向，像个底座)
# N 原子
conf_nh3.SetAtomPosition(0, Geometry.Point3D(0, 0, 0))
# 3 个 H 原子 (四面体角度，键长 ~1.0)
# 简单的三角锥几何
h_offsets = [
    (0.0, 0.94, -0.38),
    (0.81, -0.47, -0.38),
    (-0.81, -0.47, -0.38)
]
for i in range(1, 4): # H 的索引是 1,2,3
    x, y, z = h_offsets[i-1]
    conf_nh3.SetAtomPosition(i, Geometry.Point3D(x, y, z))

# B. 摆放 MeI (C 在 Z=3.0, I 在 Z=5.2)
c_idx = [a.GetIdx() for a in mol_mei.GetAtoms() if a.GetSymbol() == 'C'][0]
i_idx = [a.GetIdx() for a in mol_mei.GetAtoms() if a.GetSymbol() == 'I'][0]
h_indices = [a.GetIdx() for a in mol_mei.GetAtoms() if a.GetSymbol() == 'H']

# C 稍微偏一点点 (0.1) 避开 180 度死角
conf_mei.SetAtomPosition(c_idx, Geometry.Point3D(0.1, 0, 3.0)) 
conf_mei.SetAtomPosition(i_idx, Geometry.Point3D(0.2, 0, 5.2))

# C 上的 H (伞状，指向 N)
h_offsets_c = [(0.1, 1.03, 2.7), (0.99, -0.51, 2.7), (-0.79, -0.51, 2.7)]
for idx, off in zip(h_indices, h_offsets_c):
    conf_mei.SetAtomPosition(idx, Geometry.Point3D(off[0], off[1], off[2]))

# 合并
nh3_sys = Chem.CombineMols(mol_nh3, mol_mei)

# 自动获取序号
n_idx = [a.GetIdx()+1 for a in nh3_sys.GetAtoms() if a.GetSymbol() == 'N'][0]
c_idx = [a.GetIdx()+1 for a in nh3_sys.GetAtoms() if a.GetSymbol() == 'C'][0]
i_idx = [a.GetIdx()+1 for a in nh3_sys.GetAtoms() if a.GetSymbol() == 'I'][0]

print(f">>> 对齐完成！N={n_idx}, C={c_idx}, I={i_idx}")
view_with_labels(nh3_sys, "NH3 + MeI Aligned")

In [ ]:
# --- 3. 快速预览扫描 (Coarse Scan) ---
print(f"\n>>> 2. 启动快速预览扫描 (Preview Scan)")
print(f"    策略：步长加大，步数减少，只求看个趋势")

# 扫描 15 步，每步缩短 0.1 埃 (总共缩短 1.5 埃，足够覆盖过渡态)
scan_lines = [
    f"{n_idx} {c_idx} S 15 -0.10", 
    # f"{n_idx} {c_idx} {i_idx} F",  # 注：移除了角度冻结(F)，以避免 N-C-I 共线时的数学奇异性报错。
]

gjf_preview = create_advanced_scan_gaussian_input(
    nh3_sys,
    "NH3_Preview",
    scan_lines=scan_lines,
    charge=0, # 整体电荷 0
    nproc=6,
    mem="4GB",
    method="#PM7 opt(modredundant)", # 极速模式
    filename="nh3_preview.gjf"
)

# 运行 (这个应该非常快，1分钟左右)
run_gaussian_job(gjf_preview)

# --- 4. 检查结果 ---
print("\n>>> 3. 检查能量曲线...")
points = read_scan_output("nh3_preview.log")

if points:
    energies = [p['energy'] for p in points]
    max_e = max(energies)
    max_idx = energies.index(max_e)
    
    # 打印简易曲线
    print(f"最高能量点在第 {points[max_idx]['step']} 步")
    
    # 简单的文本绘图
    min_e = min(energies)
    print("能量趋势图 (文本版):")
    for i, e in enumerate(energies):
        # 归一化到 0-10 颗星
        stars = int((e - min_e) / (max_e - min_e) * 20)
        marker = "<-- TS?" if i == max_idx else ""
        print(f"Step {i+1:2d}: {'*' * stars} {marker}")
        
    print(f"\n>>> 建议：")
    if 0 < max_idx < len(points)-1:
        print("✅ 成功！能量先升后降，这是一个完美的过渡态曲线！")
        print("您可以直接提取第 {} 步的结构去做 TS 优化了。".format(points[max_idx]['step']))
    else:
        print("⚠️ 警告：能量没有出现峰值（一直上升或一直下降）。")
        print("可能需要调整距离范围。")

In [ ]:
# 1. 确保我们有预览扫描的结果
if 'points' in locals() and points:
    # 找到最高点
    max_e = max([p['energy'] for p in points])
    max_point = [p for p in points if p['energy'] == max_e][0]
    
    print(f">>> 提取 Preview Scan 中第 {max_point['step']} 步的结构作为初猜...")
    ts_guess_nh3, _ = extract_scan_point_to_mol(points, max_point['step'], nh3_sys, show_3d=False)
    
    # 2. 生成 TS 优化文件
    # 必须加 calcall (算力常数) 和 noeig (不打印海森矩阵特征向量)
    gjf_ts_nh3 = create_gaussian_input_advanced(
        ts_guess_nh3,
        "NH3_TS_Final",
        charge=0,
        nproc=6,
        mem="4GB",
        method="#PM7 opt(ts,noeig,calcall)", 
        filename="nh3_ts.gjf"
    )
    
    # 3. 运行
    print(">>> 正在运行 TS 优化 (Berny 算法)...")
    run_gaussian_job(gjf_ts_nh3)
    
    # 4. 读取结果并验证虚频
    results_ts_nh3 = read_gaussian16_output_opt("nh3_ts.log")
    ts_final_nh3 = update_mol_coordinates(ts_guess_nh3, results_ts_nh3)
    
    freqs = results_ts_nh3.get('frequencies')
    print(f"\n>>> 频率检查: {freqs}")
    
    if freqs and freqs[0] < 0:
        print(f"✅ 漂亮！找到唯一的虚频: {freqs[0]:.2f} cm^-1")
        print("这证实了这是一个真正的过渡态！")
        view_with_labels(ts_final_nh3, "NH3 + MeI Transition State")
    else:
        print("❌ 失败：没有找到虚频。")
else:
    print("请先运行 Preview Scan 代码块。")

In [ ]:
# ==========================================
# 阶段四：IRC 反应路径验证
# ==========================================

# 1. 确保我们有优化好的过渡态结构
if 'ts_final_nh3' in locals() and ts_final_nh3:
    print(">>> 启动 IRC (内禀反应坐标) 计算...")
    print("    目的：验证过渡态连接了正确的反应物和产物。")
    
    # 2. 生成 IRC 输入文件
    # maxpoints=15: 向每个方向走15步 (通常足够看清趋势了)
    # stepsize=10: 每步走多远 (单位是 0.1 amu^(1/2)*Bohr)
    # calcall: 为了保证 IRC 走得准，每步算 Hessian (PM7 算得起)
    gjf_irc = create_gaussian_input_advanced(
        ts_final_nh3, # 使用刚刚优化好的 TS 结构作为起点
        "NH3_IRC",
        charge=0,
        nproc=6,
        mem="4GB",
        method="#PM7 IRC(calcall, maxpoints=15, stepsize=10)", 
        filename="nh3_irc.gjf"
    )
    
    # 3. 运行 (可能需要 5-10 分钟，取决于步数)
    run_gaussian_job(gjf_irc)
    
    # 4. 解析结果
    print("\n>>> IRC 计算结束，正在解析路径...")
    irc_points = read_irc_output("nh3_irc.log")
    
    if irc_points:
        print_irc_results(irc_points)
        
        # 5. 可视化关键帧
        # 提取路径两端的终点 (Endpoint)
        # 路径1的最后一点 (通常是反应物方向)
        print("\n>>> 反应物侧终点 (Path 1 Last Point):")
        path1_end = [p for p in irc_points if p['path_number']==1][-1]
        mol_p1, _ = extract_irc_point_to_mol(irc_points, 1, path1_end['point_number'], show_3d=True)
        
        # 路径2的最后一点 (通常是产物方向)
        print("\n>>> 产物侧终点 (Path 2 Last Point):")
        path2_end = [p for p in irc_points if p['path_number']==2][-1]
        mol_p2, _ = extract_irc_point_to_mol(irc_points, 2, path2_end['point_number'], show_3d=True)
        
        print("\n>>> 结论验证：")
        print("请观察上面两个图：")
        print("图1：是否回到了 NH3 和 MeI 分开的状态？(反应物)")
        print("图2：是否变成了 Me-NH3+ 和 I- 在一起的状态？(产物)")
    else:
        print("❌ 读取 IRC 结果失败。")
else:
    print("请先成功完成 TS 优化步骤。")

In [ ]:
# ==========================================
# Cell 7: 结果可视化与键长分析 (Result Analysis)
# ==========================================
import numpy as np
from rdkit import Chem

def analyze_and_view(filename, title):
    """读取XYZ文件，计算键长，并可视化"""
    try:
        # 1. 读取分子
        mol = Chem.MolFromXYZFile(filename)
        if not mol:
            print(f"无法读取文件: {filename}")
            return

        # 2. 获取坐标 (N=idx 0, C=idx 4, I=idx 5)
        # 注意：这里的序号是基于你之前提供的XYZ文件顺序
        conf = mol.GetConformer()
        pos_n = np.array(conf.GetAtomPosition(0))
        pos_c = np.array(conf.GetAtomPosition(4))
        pos_i = np.array(conf.GetAtomPosition(5))

        # 3. 计算距离
        d_cn = np.linalg.norm(pos_n - pos_c)
        d_ci = np.linalg.norm(pos_c - pos_i)

        # 4. 打印分析结论
        print(f"\n{'='*20} {title} {'='*20}")
        print(f"关键键长分析:")
        print(f"  • C - N 距离: {d_cn:.3f} Å")
        print(f"  • C - I 距离: {d_ci:.3f} Å")
        
        # 5. 自动判定化学状态
        if d_ci < 2.3 and d_cn > 2.5:
            status = "【反应物复合物】 (Reactant Complex)"
            desc = "特征: C-I 键完整 (短)，C-N 键未形成 (长)。\n能量较低，对应中性分子 NH3 + MeI。"
        elif d_cn < 1.6 and d_ci > 3.0:
            status = "【产物离子对】 (Product Ion Pair)"
            desc = "特征: C-N 键已形成 (短)，C-I 键已断裂 (长)。\n能量较高 (吸热)，因为气相中缺乏溶剂稳定离子 (MeNH3+ ... I-)。"
        else:
            status = "【过渡态或中间区域】"
            desc = "特征: C-N 和 C-I 键长介于成键与断键之间。"
            
        print(f"化学判定: {status}")
        print(desc)
        print("-" * 50)

        # 6. 可视化
        view_with_labels(mol, title)
        
    except Exception as e:
        print(f"处理 {filename} 时出错: {e}")

# --- 执行分析 ---

# 1. 分析 Path 1 终点 (能量最低点 -> 反应物)
# 注意：文件名是你之前保存的，虽然名字叫 product，但其实是反应物
if os.path.exists("path1_end_product.xyz"):
    analyze_and_view("path1_end_product.xyz", "IRC Path 1 Endpoint (Low Energy)")
else:
    print("找不到 path1_end_product.xyz，请确保上面的保存代码已运行。")

# 2. 分析 Path 2 终点 (能量最高点 -> 产物)
if os.path.exists("path2_end_reactant.xyz"):
    analyze_and_view("path2_end_reactant.xyz", "IRC Path 2 Endpoint (High Energy)")
else:
    print("找不到 path2_end_reactant.xyz")